In [6]:
# Importation des bibliothèques
import numpy as np        # Manipulation de tableaux numériques
import pandas as pd       # Analyse et traitement de données
import matplotlib.pyplot as plt  # Visualisation de données
from pathlib import Path  # pour parcourir tous les fichiers dans le dossier.
from lxml import etree #lecture et analyse de fichier .TCX
import folium
from IPython.display import IFrame
from folium.plugins import HeatMap
from dataclasses import dataclass
import gpxpy
import json
from dataclasses import asdict
import os
from typing import Union





In [7]:
# Chemin vers le dossier contenant les fichiers TCX
tcx_folder = Path(r"C:\Users\shumb\OneDrive\Documents\Visual Studio Code\Running Datas\TCX")

# Liste tous les fichiers .tcx dans le dossier
tcx_files = list(tcx_folder.glob("*.tcx"))

print(f"Nombre de fichiers TCX trouvés : {len(tcx_files)}")
print(f"Liste des fichiers TCX : {[file.name for file in tcx_files]}")
# Namespace pour les fichiers TCX
ns = {
    'tcx': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2',
    'ns3': 'http://www.garmin.com/xmlschemas/ActivityExtension/v2'
}

Nombre de fichiers TCX trouvés : 15
Liste des fichiers TCX : ['activity_17482399352.tcx', 'activity_17490289008.tcx', 'activity_17507109341.tcx', 'activity_17512456424.tcx', 'activity_17522554619.tcx', 'activity_17535799744.tcx', 'activity_17550087476.tcx', 'activity_17556631216.tcx', 'activity_17556631939.tcx', 'activity_17649066143.tcx', 'activity_17665683882.tcx', 'activity_17686792876.tcx', 'activity_17700789297.tcx', 'activity_17715922870.tcx', 'activity_19112217557.tcx']


In [ ]:
#Convertir le temps en format lisible : Si le temps est en secondes, vous pouvez le convertir en format hh:mm:ss
import datetime

df["TimeFormatted"] = df["TotalTimeSeconds"].apply(
    lambda x: str(datetime.timedelta(seconds=x)) if pd.notnull(x) else None
)


In [8]:
@dataclass
class PointGPS:
    time: float
    latitude: float
    longitude: float
    altitude: float
    distance: float
    heartRate: int
    speed: float
    runCadence: int
    watts: int

In [9]:
@dataclass
class Lap:
    start_time: str
    total_time: float
    distance: float
    max_speed: float
    calories: int
    avg_hr: int
    max_hr: int
    intensity: str
    trigger: str
    points: list[PointGPS]


In [10]:
@dataclass
class Activite:
    sport: str
    id: str
    laps: list[Lap]

In [11]:
def get_text(elem, default='N/A', cast_func=str):
    if elem is not None and elem.text:
        try:
            return cast_func(elem.text)
        except:
            return default
    return default


In [32]:
#root : C'est l'objet racine de l'arbre XML, obtenu en analysant le fichier avec lxml (via etree.parse() ou similaire). Il représente la totalité du document XML.
#xpath() : méthode de lxml qui permet de naviguer dans la structure XML en utilisant des requêtes XPath pour sélectionner des parties spécifiques d'un document XML.
#"//tcx:Activity" : requête XPath pour trouver toutes les balises <Activity> dans le document, avec le préfixe tcx.
#namespaces=ns :associe le préfixe tcx à l'espace de noms spécifié dans le dictionnaire ns, afin que lxml puisse reconnaître les balises <Activity> qui appartiennent à cet espace de noms.
# requête : root.xpath("//tcx:Activity", namespaces=ns)
#Le double slash // signifie "sélectionne tous les nœuds correspondants, quel que soit leur emplacement dans le document".
#Cela permet de rechercher les balises <Activity> où qu'elles se trouvent, à n'importe quel niveau de la hiérarchie XML.

def parse_tcx_file(file_path: str) -> Activite:
    ns = {
        "tcx": "http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2",
        "ns3": "http://www.garmin.com/xmlschemas/ActivityExtension/v2"
    }

    with open(file_path, 'rb') as f:
        root = etree.parse(f).getroot()

    activity = root.find(".//tcx:Activity", namespaces=ns)
    if activity is None:
        raise ValueError(f"Aucune activité trouvée dans le fichier {file_path}")

    sport = activity.get("Sport", "N/A")
    activity_id = Path(file_path).stem  # Nom du fichier sans extension

    laps = []
    for lap in activity.findall("tcx:Lap", namespaces=ns):
        start_time = lap.attrib.get("StartTime", "N/A")
        total_time = get_text(lap.find("tcx:TotalTimeSeconds", namespaces=ns), 0.0, float)
        distance = get_text(lap.find("tcx:DistanceMeters", namespaces=ns), 0.0, float)
        max_speed = get_text(lap.find("tcx:MaximumSpeed", namespaces=ns), 0.0, float)
        calories = get_text(lap.find("tcx:Calories", namespaces=ns), 0, int)
        avg_hr = get_text(lap.find("tcx:AverageHeartRateBpm/tcx:Value", namespaces=ns), 0, int)
        max_hr = get_text(lap.find("tcx:MaximumHeartRateBpm/tcx:Value", namespaces=ns), 0, int)
        intensity = get_text(lap.find("tcx:Intensity", namespaces=ns))
        trigger = get_text(lap.find("tcx:TriggerMethod", namespaces=ns))

        points = []
        for trkpt in lap.findall(".//tcx:Trackpoint", namespaces=ns):
            time = get_text(trkpt.find("tcx:Time", namespaces=ns))
            lat = get_text(trkpt.find("tcx:Position/tcx:LatitudeDegrees", namespaces=ns), 0.0, float)
            lon = get_text(trkpt.find("tcx:Position/tcx:LongitudeDegrees", namespaces=ns), 0.0, float)
            alt = get_text(trkpt.find("tcx:AltitudeMeters", namespaces=ns), 0.0, float)
            dist = get_text(trkpt.find("tcx:DistanceMeters", namespaces=ns), 0.0, float)
            hr = get_text(trkpt.find("tcx:HeartRateBpm/tcx:Value", namespaces=ns), 0, int)

            ext = trkpt.find("tcx:Extensions", namespaces=ns)
            speed = cadence = watts = 0
            if ext is not None:
                speed = get_text(ext.find(".//ns3:Speed", namespaces=ns), 0.0, float)
                cadence = get_text(ext.find(".//ns3:RunCadence", namespaces=ns), 0, int)
                watts = get_text(ext.find(".//ns3:Watts", namespaces=ns), 0, int)

            points.append(PointGPS(time, lat, lon, alt, dist, hr, speed, cadence, watts))

        laps.append(Lap(start_time, total_time, distance, max_speed, calories, avg_hr, max_hr, intensity, trigger, points))

    return Activite(sport, activity_id, laps)


In [ ]:
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # rayon de la Terre en mètres
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    d_phi = math.radians(lat2 - lat1)
    d_lambda = math.radians(lon2 - lon1)

    a = math.sin(d_phi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(d_lambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c  # en mètres

In [62]:
import json
import os
from pathlib import Path
from dataclasses import asdict

ACTIVITIES_DIR = Path(r"C:\Users\shumb\OneDrive\Documents\Visual Studio Code\Running Datas\site\activities_json")
INDEX_PATH = ACTIVITIES_DIR / "index.json"

# Chargement ou création de l'index
def load_index(index_path: Path) -> list[str]:
    if index_path.exists():
        with open(index_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    return []

def save_index(index: list[str], index_path: Path):
    with open(index_path, 'w', encoding='utf-8') as f:
        json.dump(index, f, indent=2)

# Vérifie si un fichier TCX a déjà été traité
def tcx_already_parsed(tcx_filename: str, index: list[str]) -> bool:
    return tcx_filename in index

# Sauvegarde une activité en JSON
def save_activity_to_json(activity, json_path: Path):
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(asdict(activity), f, indent=2)

def convert_all_tcx_to_json(tcx_dir: Path, json_dir: Path, index_path: Path):
    json_dir.mkdir(parents=True, exist_ok=True)
    index_path.parent.mkdir(parents=True, exist_ok=True)

    index = load_index(index_path)

    for file in Path(tcx_dir).glob("*.tcx"):
        if tcx_already_parsed(file.name, index):
            print(f"Fichier déjà traité : {file.name}")
            continue

        print(f"Traitement de {file.name}")
        activity = parse_tcx_file(file)

        activity_id = Path(file.name).stem  # nom sans extension
        output_file = json_dir / f"{activity_id}.json"
        save_activity_to_json(activity, output_file)

        index.append(f"{activity_id}.json")  # Ajoute le nom du fichier JSON à l'index

    save_index(index, index_path)

# Exemple d'utilisation
tcx_dir = Path(r"C:\Users\shumb\OneDrive\Documents\Visual Studio Code\Running Datas\site\tcx_files")
json_dir = ACTIVITIES_DIR
index_path = INDEX_PATH

convert_all_tcx_to_json(tcx_dir, json_dir, index_path)


In [63]:
def convert_all_tcx_to_json(tcx_dir: Path, json_dir: Path, index_path: Path):
    json_dir.mkdir(parents=True, exist_ok=True)
    index_path.parent.mkdir(parents=True, exist_ok=True)

    index = load_index(index_path)

    for file in Path(tcx_dir).glob("*.tcx"):
        if tcx_already_parsed(file.name, index):
            print(f"Fichier déjà traité : {file.name}")
            continue

        print(f"Traitement de {file.name}")
        activity = parse_tcx_file(file)

        activity_id = Path(file.name).stem  # nom sans extension
        output_file = json_dir / f"{activity_id}.json"
        save_activity_to_json(activity, output_file)

        index.append(f"{activity_id}.json")  # Ajoute le nom du fichier JSON à l'index

    save_index(index, index_path)



In [66]:
# Exemple d'utilisation
tcx_dir = Path(r"C:\Users\shumb\OneDrive\Documents\Visual Studio Code\Running Datas\TCX")
json_dir = ACTIVITIES_DIR
index_path = INDEX_PATH

convert_all_tcx_to_json(tcx_dir, json_dir, index_path)


Traitement de activity_17482399352.tcx
Traitement de activity_17490289008.tcx
Traitement de activity_17507109341.tcx
Traitement de activity_17512456424.tcx
Traitement de activity_17522554619.tcx
Traitement de activity_17535799744.tcx
Traitement de activity_17550087476.tcx
Traitement de activity_17556631216.tcx
Traitement de activity_17556631939.tcx
Traitement de activity_17649066143.tcx
Traitement de activity_17665683882.tcx
Traitement de activity_17686792876.tcx
Traitement de activity_17700789297.tcx
Traitement de activity_17715922870.tcx
Traitement de activity_19112217557.tcx
